In [24]:
from happytransformer import HappyTextToText
from happytransformer import TTSettings
from datasets import load_dataset
import csv
from happytransformer import TTTrainArgs

In [2]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

03/05/2022 16:05:13 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [4]:
args = TTSettings(num_beams=5, min_length=1)

In [11]:
result = happy_tt.generate_text("grammar: She see Tom is catched by policeman in park at last night ", args=args)

In [12]:
result.text

'She sees Tom is catched by a policeman in a park last night.'

In [14]:
train_dataset = load_dataset("jfleg", split='validation[:]')

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset jfleg downloaded and prepared to C:\Users\prato\.cache\huggingface\datasets\jfleg\default\1.0.0\ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b. Subsequent calls will reuse this data.


In [15]:
eval_dataset = load_dataset("jfleg", split='test[:]')

In [20]:
for case in train_dataset["corrections"][:2]:
    print(case)
    print(case[0])

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 


In [19]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                if input_text and correction:
                    writter.writerow([input_text, correction])

generate_csv("D:/train.csv", train_dataset)
generate_csv("D:/eval.csv", eval_dataset)

In [21]:
before_result = happy_tt.eval("eval.csv")

03/05/2022 16:53:25 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\prato\.cache\huggingface\datasets\csv\default-f19c444f80ff2f50\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


In [22]:
print(before_result.loss)

0.5190436840057373


In [25]:
args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

03/05/2022 16:59:33 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\prato\.cache\huggingface\datasets\csv\default-a7a23fc07b8f55d9\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

03/05/2022 16:59:38 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
C:\Users\prato\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1131


Step,Training Loss
500,0.398400
1000,0.332800




Training completed. Do not forget to share your model on huggingface.co/models =)




In [30]:
from happytransformer import TTSettings
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)
example_2 = "grammar: I am enjoys, writtings articles ons AI."

result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoy writing articles on AI.


In [31]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
    for rep in replacements:
        text = text.replace(rep[0], rep[1])
    return text